In [1]:
import ee
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from topojson import Topology

import agrigee_lite as agl

np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)
pd.set_option('display.float_format', '{:.2f}'.format)

# pandas show all columns
pd.set_option('display.max_columns', None)

In [2]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="ee-paulagibrim")

In [3]:
gdf = gpd.read_parquet("tests/data/gdf.parquet")

In [4]:
satellite = agl.sat.Sentinel2()

In [7]:
row = gdf.iloc[0]

sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite, ["skew"])

In [12]:
sits.s2_green_39

0   0.44
Name: s2_green_39, dtype: float64

In [9]:
sits2 = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite, ["skew"])

In [11]:
sits2.s2_green_39

0   0.44
Name: s2_green_39, dtype: float64

In [ ]:
agl.vis.sits(row.geometry, row.start_date, row.end_date, satellite, "blue")

In [ ]:
from agrigee_lite.misc import wide_to_long_dataframe

In [ ]:
long_sits = wide_to_long_dataframe(sits)

In [ ]:
long_sits["evi2"] = 2.4 * (long_sits["nir"] - long_sits["red"]) / (long_sits["nir"] + 2.4 * long_sits["red"] + 1)

In [ ]:
long_sits

In [ ]:
from agrigee_lite.numpy_indices import ALL_NUMPY_INDICES

In [ ]:
def single_sits(geometry, start_date, end_date, satellite, band_or_indice_to_plot):
    sits = agl.get.sits(geometry, start_date, end_date, satellite)
    long_sits = wide_to_long_dataframe(sits)

    return plt.plot(long_sits.doy, long_sits[band_or_indice_to_plot])

In [ ]:
def single_sits(geometry, start_date, end_date, satellite, band_or_indice_to_plot):
    sits = agl.get.sits(geometry, start_date, end_date, satellite)
    long_sits = wide_to_long_dataframe(sits)

    return plt.plot(long_sits.doy, long_sits[band_or_indice_to_plot])

In [ ]:
ALL_NUMPY_INDICES

In [ ]:
compute_index_from_df(long_sits, ALL_NUMPY_INDICES["ndvi"])

In [ ]:
satellite.scaleBands()

In [ ]:
single_sits(row.geometry, row.start_date, row.end_date, satellite, "blue")

In [ ]:
df_pivot

In [ ]:
plt.plot(df_pivot.doy, df_pivot.nir)

In [ ]:
df_pivot.sort_values(by=["doy"], inplace=True)

In [ ]:
gdf = agl.quadtree_clustering(gdf)

In [ ]:
gdf[gdf.cluster_id==10][0:1].plot()

In [ ]:
topo = Topology(gdf[gdf.cluster_id==100], prequantize=False)
topo = topo.toposimplify(0.001, prevent_oversimplify=True)
simple_gdf = topo.to_gdf()

In [ ]:
_, ax = plt.subplots(1,2)

gdf[gdf.cluster_id==100].plot(ax=ax[0])
simple_gdf.plot(ax=ax[1])

In [ ]:
_, ax = plt.subplots(1,2)

gdf[gdf.cluster_id==100][3:4].plot(ax=ax[0])
simple_gdf[3:4].plot(ax=ax[1])

In [ ]:
gdf[gdf.cluster_id==100][0:1].to_crs("EPSG:3857").area/10000

In [ ]:
gdf[gdf.cluster_id==100].geometry.astype(str).apply(len).sum(), simple_gdf.geometry.astype(str).apply(len).sum()

In [ ]:
simplified_gdf[0:1].plot()

In [ ]:
gdf[gdf.cluster_id==10].geometry.astype(str).apply(len)

In [ ]:
simplified_gdf.geometry.astype(str).apply(len)

In [ ]:
satellite = agl.sat.Sentinel2()

In [ ]:
agl.get.multiple_sits_chunks_gdrive(gdf, satellite)

In [ ]:
from agrigee_lite.misc import create_gdf_hash

In [ ]:
hashname = create_gdf_hash(gdf)

In [ ]:
satellite = agl.sat.Sentinel2(use_sr=True, selected_bands=["red", "green", "blue"])

In [ ]:
for cluster_id in sorted(gdf.cluster_id.unique()):
    cluster_id = int(cluster_id)
    agl.get.multiple_sits_task_gdrive(gdf[gdf.cluster_id == cluster_id], satellite, f"{hashname}_{cluster_id}", f"agl_mateus_multiple_sits_{hashname}_{cluster_id}")

    if cluster_id > 5:
        break

In [ ]:
row = gdf.iloc[0]

In [ ]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
sits

In [ ]:
sits_2

In [ ]:
df_wide = pd.DataFrame(sits_2.values.flatten().reshape(1, -1))
df_wide.columns = [f"{col}_{i}" for i in range(len(sits_2)) for col in sits_2.columns]

In [ ]:
sits = agl.get.multiple_sits(gdf.iloc[0:2], satellite)

In [ ]:
def flatten_by_indexnum(df: pd.DataFrame, prefix: str = "") -> pd.DataFrame:
    if "indexnum" not in df.columns:
        raise ValueError("A coluna 'indexnum' não existe no DataFrame.")

    df = df.copy()
    df["seq"] = df.groupby("indexnum").cumcount()
    df_wide = df.pivot(index="indexnum", columns="seq").drop(columns="indexnum")
    df_wide.columns = [f"{prefix}{col}_{i}" for col, i in df_wide.columns]
    return df_wide.reset_index()

In [ ]:
sits

In [ ]:
flatten_by_indexnum(sits)

In [ ]:
flatten_by_indexnum(sits, "a")

In [ ]:
row = gdf.iloc[0]

imgs = agl.get.images(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
np.savez_compressed("tests/data/imgs/0_s2sr.npz", data=imgs)

In [ ]:
np.load("tests/data/imgs/0_s2sr.npz")

In [ ]:
np.array_equal(imgs, imgs)

In [ ]:
imgs_copy = imgs.copy()

In [ ]:
plt.imshow((imgs[3]/10000) * np.pi)

In [ ]:
sits = agl.get.sits(row.geometry, row.start_date, row.end_date, satellite)

In [ ]:
sits.columns = [column.split("_", 1)[1] for column in sits.columns.tolist()]

In [ ]:
def remove_underscore_in_df(df):
    df.columns = [column.split("_", 1)[1] for column in df.columns.tolist()]

In [ ]:
sits.head(5)